In [1]:
import configparser
import boto3

In [2]:
import logging
logging.basicConfig(format='[ %(asctime)s ] %(filename)s(%(lineno)d) %(levelname)s - %(message)s', level=logging.INFO)

In [14]:
logger = logging.getLogger()

In [3]:
%load_ext autoreload
%autoreload 2

In [5]:
import configparser

In [6]:
config = configparser.ConfigParser()

In [4]:
config.read('aws.cfg')

['aws.cfg']

In [6]:
KEY=config.get('AWS','key')
SECRET= config.get('AWS','secret')
REGION = 'us-west-2'

In [22]:
s3 = boto3.resource('s3',
    aws_access_key_id=KEY,
    aws_secret_access_key=SECRET)

In [23]:
bucket = s3.Bucket('udacity-dend')

In [25]:
bucket.download_file('log_json_path.json', 'log_data_path.json')

In [8]:
iam = boto3.resource(
    'iam',
    aws_access_key_id=KEY,
    aws_secret_access_key=SECRET
)

In [10]:
type(iam)

boto3.resources.factory.iam.ServiceResource

In [47]:
for policy in d.attached_policies

[iam.Policy(arn='arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess')]

In [16]:
from create_infra import *

In [8]:
aws_config = configparser.ConfigParser()
aws_config.read('aws.cfg')
KEY=aws_config.get('AWS','key')
SECRET= aws_config.get('AWS','secret')

dwh_config = configparser.ConfigParser()
dwh_config.read('dwh.cfg')
DWH_ROLE_NAME = dwh_config.get("IAM_ROLE", "ROLE_NAME")

REGION                 = dwh_config.get("CLUSTER_CONFIG", "REGION")
DWH_CLUSTER_TYPE       = dwh_config.get("CLUSTER_CONFIG","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = dwh_config.get("CLUSTER_CONFIG","DWH_NUM_NODES")
DWH_NODE_TYPE          = dwh_config.get("CLUSTER_CONFIG","DWH_NODE_TYPE")
DWH_CLUSTER_IDENTIFIER = dwh_config.get("CLUSTER_CONFIG","DWH_CLUSTER_IDENTIFIER")

DWH_DB                 = dwh_config.get("CLUSTER","DB_NAME")
DWH_DB_USER            = dwh_config.get("CLUSTER","DB_USER")
DWH_DB_PASSWORD        = dwh_config.get("CLUSTER","DB_PASSWORD")
DWH_PORT               = dwh_config.get("CLUSTER","DB_PORT")

iam = boto3.resource(
    'iam',
    aws_access_key_id=KEY,
    aws_secret_access_key=SECRET
)
redshift = boto3.client(
    'redshift', 
    REGION,
    aws_access_key_id=KEY,
    aws_secret_access_key=SECRET
)

In [20]:

# roles are lightweight objects to create/destroy, so if the role 
# already exists, delete it and recreate it to make sure that the policies
# are correct
if iam_role_exists(iam, DWH_ROLE_NAME):
    logger.info(f"Role {DWH_ROLE_NAME} already exists")
    delete_iam_role(iam, DWH_ROLE_NAME)
else:
    logger.info(f"Role {DWH_ROLE_NAME} doesn't exist")
role_arn = create_iam_role(
    iam, DWH_ROLE_NAME,
    "redshift.amazonaws.com", 
    'arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess'
)


[ 2020-05-24 13:13:54,289 ] <ipython-input-20-6e042e1629b1>(6) INFO - Role sparkify_dwh_role already exists
[ 2020-05-24 13:13:54,293 ] aws_iac.py(32) INFO - Deleting the role sparkify_dwh_role and its attached policies
[ 2020-05-24 13:13:54,491 ] aws_iac.py(46) INFO - Creating Role sparkify_dwh_role with arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess policy


In [24]:
from aws_iac import *

In [23]:

# Redshift clusters are not that fast to create and we don't want to lose
# whatever data we have in an existing cluster, so if a cluster exists, 
# we'll just use that
if redshift_cluster_exists(redshift, DWH_CLUSTER_IDENTIFIER):
    logger.info(f"Redshift Cluster {DWH_CLUSTER_IDENTIFIER} already exists, using that one")
    redshift_host = get_redshift_cluster_endpoint(redshift, DWH_CLUSTER_IDENTIFIER)
else:
    redshift_host = create_redshift_cluster(
        redshift, role_arn,
        DWH_CLUSTER_TYPE, DWH_NODE_TYPE,
        int(DWH_NUM_NODES),
        DWH_CLUSTER_IDENTIFIER, 
        DWH_DB, int(DWH_PORT),
        DWH_DB_USER, DWH_DB_PASSWORD
    )


[ 2020-05-24 13:19:45,059 ] <ipython-input-23-dae729aa38ef>(6) INFO - Redshift Cluster sparkify-dwh-cluster already exists, using that one


In [26]:
delete_redshift_cluster(redshift, DWH_CLUSTER_IDENTIFIER)

[ 2020-05-24 13:21:42,379 ] aws_iac.py(133) INFO - Deleting the redshift cluster sparkify-dwh-cluster
[ 2020-05-24 13:21:42,733 ] aws_iac.py(138) INFO - Waiting for the cluster to be deleted....
[ 2020-05-24 13:24:44,102 ] aws_iac.py(143) INFO - Cluster deleted successfully


In [ ]:
redshift.describe_clusters

In [24]:
role = iam.Role('dwhRole')

In [21]:
iam.list_attached_role_policies(RoleName='dwhRole')

{'AttachedPolicies': [{'PolicyName': 'AmazonS3ReadOnlyAccess',
   'PolicyArn': 'arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess'}],
 'IsTruncated': False,
 'ResponseMetadata': {'RequestId': '72f58b04-4e39-4738-94c0-59185fe94102',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '72f58b04-4e39-4738-94c0-59185fe94102',
   'content-type': 'text/xml',
   'content-length': '548',
   'date': 'Sun, 24 May 2020 10:52:10 GMT'},
  'RetryAttempts': 0}}

In [19]:
iam

{'Role': {'Path': '/',
  'RoleName': 'dwhRole',
  'RoleId': 'AROATGEAZHAIQ4VXBZ5Z6',
  'Arn': 'arn:aws:iam::219313420305:role/dwhRole',
  'CreateDate': datetime.datetime(2020, 5, 10, 10, 39, 1, tzinfo=tzlocal()),
  'AssumeRolePolicyDocument': {'Version': '2012-10-17',
   'Statement': [{'Effect': 'Allow',
     'Principal': {'Service': 'redshift.amazonaws.com'},
     'Action': 'sts:AssumeRole'}]},
  'MaxSessionDuration': 3600},
 'ResponseMetadata': {'RequestId': 'f0b38272-c8f2-49b9-b55a-65c54809b694',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f0b38272-c8f2-49b9-b55a-65c54809b694',
   'content-type': 'text/xml',
   'content-length': '919',
   'date': 'Sun, 24 May 2020 10:47:34 GMT'},
  'RetryAttempts': 0}}

In [31]:
all_p = d.policies.all()

In [38]:
for p in d.policies.all():
    print('lskdfjs')

In [7]:

redshift = boto3.client(
    'redshift', 
    REGION,
    aws_access_key_id=KEY,
    aws_secret_access_key=SECRET
)